## Process SG Coarse Dataset and Digest ONET CIP and SOC ID

In [1]:
!pip install pandas sklearn
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post5-py3-none-any.whl size=2950 sha256=1bb24d991f0bab962ca5c5c53778b9156f169dca14ae1073e7708ff99117f94f
  Stored in directory: /root/.cache/pip/wheels/38/1f/8d/4f812c590e074c1e928f5cec67bf5053b71f38e2648739403a
Successfully built sklearn
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Import the necessary packages
import pandas as pd
import re
from operator import itemgetter
import json


In [3]:
# Load the data into a pandas dataframe:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Note: You will need to create a shortcut for the "MTECH IS Project" shared folder on your MyDrive to be able to access it directly.
!ls 'content/drive/MyDrive/MTECH IS Project/data/'
!ls '/content/drive/MyDrive/MTECH IS Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv'
!ls '/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_instructional_programs.csv'
!ls '/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_SOC2018.csv'

ls: cannot access 'content/drive/MyDrive/MTECH IS Project/data/': No such file or directory
'/content/drive/MyDrive/MTECH IS Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv'
'/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_instructional_programs.csv'
'/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_SOC2018.csv'


In [5]:
# load the SG Coarse Dataset
# file = "/content/drive/MyDrive/MTECH IS Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv"
file = "/content/drive/MyDrive/MTECH IS Project/data/edu_courses_from_db.csv"
# file = "/content/drive/MyDrive/MTECH\ IS\ Project/data/course_graduate_employment/graduate-employment-survey-ntu-nus-sit-smu-suss-sutd.csv"
sg_df = pd.read_csv(file, encoding = 'unicode_escape', engine ='python', index_col="id")
# sg_df = pd.read_csv(file, engine ='python')
sg_df.head()

,university,school,degree,created_at,updated_at
id,,,,,
1,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800
2,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800
3,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800
4,Nanyang Technological University,College of Business (Nanyang Business School),Business and Computing,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800
5,Nanyang Technological University,College of Engineering,Aerospace Engineering,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800


In [6]:
len(sg_df['degree'].unique())

312

In [7]:
# load the Onet Coarse Instructural Program (CIP) Dataset
# file = "/content/drive/My Drive/data_processing/CIP2020_instructional_programs.csv"
file = "/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_instructional_programs.csv"
cip_df = pd.read_csv(file,encoding = 'utf-8', dtype=str)

index_col = "CIP2020Code"

cip_df.head(3)

,CIP2020Code,CIP2020Title
0,01.0000,"Agriculture, General."
1,01.0101,"Agricultural Business and Management, General."
2,01.0102,Agribusiness/Agricultural Business Operations.


In [8]:
new_cip_df = cip_df.assign(id=cip_df['CIP2020Code'], name=cip_df['CIP2020Title'])[['id', 'name']]
new_cip_df.head()

,id,name
0,01.0000,"Agriculture, General."
1,01.0101,"Agricultural Business and Management, General."
2,01.0102,Agribusiness/Agricultural Business Operations.
3,01.0103,Agricultural Economics.
4,01.0104,Farm/Farm and Ranch Management.


In [9]:
new_cip_df = new_cip_df.set_index('id', drop=False)
new_cip_df_json = new_cip_df.to_json('cips.json', orient='index', index=True)

In [10]:
# load the CIP to SOC equivalent
# file = "/content/drive/My Drive/data_processing/CIP2020_SOC2018.csv"
file = "/content/drive/MyDrive/MTECH IS Project/data/Education/CIP2020_SOC2018.csv"
soc_df = pd.read_csv(file,encoding = 'utf-8', dtype=str)
soc_df.head(3)
#soc_df.head()
# soc_df.columns = ['CIP2020Code', 'SOC2018Code']
# soc_df.head()

,CIP2020Code,SOC2018Code
0,01.0000,19-1011
1,01.0000,19-1012
2,01.0000,19-1013


In [11]:
# Merging Onet Education Dataset
onet_soc_cip_df = cip_df.merge(
  soc_df, on=index_col
)

onet_soc_cip_df.head(10)

,CIP2020Code,CIP2020Title,SOC2018Code
0,01.0000,"Agriculture, General.",19-1011
1,01.0000,"Agriculture, General.",19-1012
2,01.0000,"Agriculture, General.",19-1013
3,01.0000,"Agriculture, General.",19-4012
4,01.0000,"Agriculture, General.",25-1041
5,01.0101,"Agricultural Business and Management, General.",11-9013
6,01.0101,"Agricultural Business and Management, General.",25-1041
7,01.0101,"Agricultural Business and Management, General.",45-1011
8,01.0102,Agribusiness/Agricultural Business Operations.,11-9013
9,01.0102,Agribusiness/Agricultural Business Operations.,25-1041


In [12]:
# Match CIP to singapore courses
!pip install pandas sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.8 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4e8141f96cbfdd4d4a74278cd4d56ef0a5900f9c81e297d18c1dd17ba5206099
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [13]:
import re
import json
from scipy import spatial
from sentence_transformers import SentenceTransformer

In [14]:
model = SentenceTransformer("/content/drive/MyDrive/MTECH IS Project/data/model")

In [15]:
# Calculate embeddings
cip_df['embeddings'] = cip_df['CIP2020Title'].apply(lambda x: model.encode(x))

cip_df.head()

,CIP2020Code,CIP2020Title,embeddings
0,01.0000,"Agriculture, General.","[0.01568099, 0.040816717, 0.006125398, 0.00662..."
1,01.0101,"Agricultural Business and Management, General.","[-0.022452775, -0.018485663, -0.009557902, -0...."
2,01.0102,Agribusiness/Agricultural Business Operations.,"[0.0012624831, -0.010626276, 0.0022416145, -0...."
3,01.0103,Agricultural Economics.,"[-0.018673254, 0.040297225, -0.0037877806, -0...."
4,01.0104,Farm/Farm and Ranch Management.,"[0.037155434, 0.018814497, -0.024712818, -0.04..."


In [16]:
sg_df['embeddings'] = sg_df['degree'].apply(lambda x: model.encode(x))
sg_df.head()

,university,school,degree,created_at,updated_at,embeddings
id,,,,,,
1,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy and Business,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800,"[-0.025065823, -0.002333231, -0.022176862, 0.0..."
2,Nanyang Technological University,College of Business (Nanyang Business School),Accountancy (3-yr direct Honours Programme),2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800,"[-0.05485546, -0.009202134, -0.0012992629, 0.0..."
3,Nanyang Technological University,College of Business (Nanyang Business School),Business (3-yr direct Honours Programme),2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800,"[-0.029972183, -0.051274773, -0.0035788673, 0...."
4,Nanyang Technological University,College of Business (Nanyang Business School),Business and Computing,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800,"[0.020817395, 0.003975899, -0.061784256, -0.01..."
5,Nanyang Technological University,College of Engineering,Aerospace Engineering,2023-05-04 23:08:01.555 +0800,2023-05-04 23:08:01.555 +0800,"[-0.0007849671, -0.021245671, -0.013078914, 0...."


In [17]:
# Calculate cosine similarity
def calc_similarity(cip_embeddings):
    sims = sg_df['embeddings'].apply(lambda sg: 1 - spatial.distance.cosine(sg, cip_embeddings))
    return list(sims[sims.gt(0.75)].index)

In [18]:
cip_df['matches'] = cip_df['embeddings'].apply(calc_similarity)

In [19]:
# Show selected
cip_df['matches_names'] = cip_df['matches'].apply(lambda x: [sg_df.loc[x]['degree'] for i in x])
cip_df['count'] = cip_df['matches'].apply(lambda x: len(x))

cip_df[['CIP2020Code', 'CIP2020Title', 'matches', 'matches_names', 'count']].loc[cip_df['count'] > 0].sample(50)

,CIP2020Code,CIP2020Title,matches,matches_names,count
1709,52.0305,Accounting and Business/Management.,"[1, 138, 160, 231, 298]","[[Accountancy and Business, Accountancy, Accou...",5
1453,51.0599,"Advanced/Graduate Dentistry and Oral Sciences,...",[48],[[Bachelor of Dental Surgery]],1
459,14.4501,Biological/Biosystems Engineering.,[6],[[Bioengineering]],1
231,11.0104,Informatics.,[294],[[Information Systems]],1
1102,40.0801,"Physics, General.","[31, 266, 335]","[[Physics / Applied Physics, Physics & Applied...",3
1215,45.0199,"Social Sciences, Other.","[17, 18, 19, 20, 21, 22, 23, 24, 37, 146, 147,...","[[Arts & Social Sciences, Arts & Social Scienc...",22
432,14.1101,Engineering Mechanics.,[15],[[Mechanical Engineering]],1
419,14.0802,Geotechnical and Geoenvironmental Engineering.,"[304, 332]","[[Environmental Earth Systems Sciences ^, Envi...",2
1112,40.1001,Materials Science.,[14],[[Materials Engineering]],1
484,15.0507,Environmental/Environmental Engineering Techno...,"[59, 137, 194, 230, 275]",[[Bachelor of Engineering (Environmental Engin...,5


In [20]:
# CIP to SG courses crosswalk
cip_programs_dict = {}

for index, row in cip_df.iterrows():
  for match_id in row["matches"]:
    id = f"{row['CIP2020Code']}__{match_id}"
    cip_programs_dict[id] = {
        "id": id,
        "program_id": match_id,
        "cip_id": row['CIP2020Code']
    }

with open('cip_programs.json', 'w') as f:
    json.dump(cip_programs_dict, f)

In [21]:
len(cip_programs_dict.keys())

321

In [27]:
# CIP to O*NET SOC
# O*NET data
occupation_data_file = "/content/drive/MyDrive/MTECH IS Project/data_processing/jobs_tasks.csv"
# occupation_data_file = "https://www.onetcenter.org/dl_files/database/db_27_2_text/Occupation%20Data.txt"
onet_df = pd.read_csv(occupation_data_file)

# onet_df = onet_df.dropna(subset=["tasks"])

onet_df['code_short'] = onet_df['O*NET-SOC Code'].str.split('.', 1).str[0]
onet_df.head()

<ipython-input-27-c09f5bd77572>:9: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  onet_df['code_short'] = onet_df['O*NET-SOC Code'].str.split('.', 1).str[0]


,O*NET-SOC Code,Title,Description,Task,embeddings,code_short
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011
1,11-1011.03,Chief Sustainability Officers,"Communicate and coordinate with management, sh...",Monitor and evaluate effectiveness of sustaina...,"[0.045658957213163376, 0.06365562975406647, 0....",11-1011
2,11-1021.00,General and Operations Managers,"Plan, direct, or coordinate the operations of ...","Review financial statements, sales or activity...","[0.03721920773386955, 0.028162039816379547, -0...",11-1021
3,11-1031.00,Legislators,"Develop, introduce, or enact laws and statutes...",Analyze and understand the local and national ...,"[0.03557248041033745, 0.05577366426587105, 0.0...",11-1031
4,11-2011.00,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici...",Plan and prepare advertising and promotional m...,"[0.08189830929040909, 0.02783762663602829, 0.0...",11-2011


In [28]:
onet_join_df = onet_df.merge(onet_soc_cip_df, left_on='code_short', right_on='SOC2018Code')

# Rename columns for DB
onet_join_df['occupation_id'] = onet_join_df['O*NET-SOC Code']
onet_join_df['cip_id'] = onet_join_df['CIP2020Code']
onet_join_df['id'] = onet_join_df['O*NET-SOC Code'] + "__" + onet_join_df['CIP2020Code']

onet_join_df.head()

,O*NET-SOC Code,Title,Description,Task,embeddings,code_short,CIP2020Code,CIP2020Title,SOC2018Code,occupation_id,cip_id,id
0,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011,44.0401,Public Administration.,11-1011,11-1011.00,44.0401,11-1011.00__44.0401
1,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011,52.0101,"Business/Commerce, General.",11-1011,11-1011.00,52.0101,11-1011.00__52.0101
2,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011,52.0201,"Business Administration and Management, General.",11-1011,11-1011.00,52.0201,11-1011.00__52.0201
3,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011,52.0206,Non-Profit/Public/Organizational Management.,11-1011,11-1011.00,52.0206,11-1011.00__52.0206
4,11-1011.00,Chief Executives,Determine and formulate policies and provide o...,Direct or coordinate an organization's financi...,"[0.04932650551199913, 0.025542698800563812, 0....",11-1011,52.0701,Entrepreneurship/Entrepreneurial Studies.,11-1011,11-1011.00,52.0701,11-1011.00__52.0701


In [29]:
onet_join_df = onet_join_df[['occupation_id', 'cip_id', 'id']]
onet_join_df = onet_join_df.set_index('id', drop=False)


In [30]:
onet_join_df.to_json('cip_occupations.json', orient='index', index=True)